Fecha: Julio de 2025

Se busca hacer análisis de frecuencias en las columnas de opinión por año, mes y periódico. E implementar modelado de tópicos. Primer método LDA.

# Librerias

In [1]:
# Librerías
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
import pyLDAvis
import pyLDAvis.lda_model
import json
import pickle
from datetime import datetime

In [2]:
# Ruta del directorio que contiene los archivos CSV
folder_path = r'C:\Users\karen\Documents\HumanidadesDigitales_git\BDD_Corpus\BDD_CSV'


In [3]:
dataframes = []

for archivo in os.listdir(folder_path):
    if archivo.endswith('.csv'):
        file_path = os.path.join(folder_path, archivo)        
        try:
            df = pd.read_csv(file_path, encoding='latin-1', sep=';')  
            dataframes.append(df)
        except Exception as e:
            print(f'Error al procesar {file_path}: {e}')


In [4]:
# Concatenar todos los DataFrames en uno solo
if dataframes:
    corpus_completo = pd.concat(dataframes, ignore_index=True)

corpus_completo.head()                                                                                                                              

,Diario,Autor,Fecha,Título,Texto,Vínculo
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...


In [5]:
# Limpiar espacios en blanco al inicio y al final de las cadenas
corpus_completo = corpus_completo.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


In [6]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

# Descripción Corpus

In [7]:
# filas que tienen fecha inválida
fechas_invalidas = corpus_completo[corpus_completo['Fecha_formateada'].isna()]
fechas_invalidas

,Diario,Autor,Fecha,Título,Texto,Vínculo,Fecha_formateada


In [8]:
autores = corpus_completo['Autor'].unique().tolist()

In [9]:
len(autores)

577

In [10]:
autores = sorted(autores)

In [11]:
print(autores)

['Abdón Espinosa Valderrama', 'Abel Veiga Copo', 'Adolfo León Atehortúa Cruz', 'Adolfo Meisel Roca', 'Adolfo Zableh Durán', 'Adriana Cooper', 'Adriana La Rotta', 'Adriana Noreña', 'Alberto Donadio', 'Alberto Galán', 'Alberto López de Mesa', 'Aldo Civico', 'Alejandro Alvarado', 'Alejandro Daly', 'Alejandro Peláez', 'Alejandro Reyes Posada', 'Alejandro Riveros González', 'Alejandro Tagliavini', 'Alexánder Cambero', 'Alfonso Carvajal', 'Alfonso Cuéllar', 'Alfonso Gómez Méndez', 'Alfonso Llano Escobar', 'Alfonso Sánchez Cadavid', 'Alfredo Molano Jimeno', 'Alister Ramírez Márquez', 'Allison Benson Hernandez', 'Allison Benson Hernández', 'Alonso Sánchez', 'Alpher Rojas C.', 'Alvaro Forero Tascón', 'Alvaro Jimenez', 'Amadeo Rodríguez Castilla', 'Ana Cristina Restrepo Jiménez', 'Ana Güezmes García', 'Ana Lucía Lenis', 'Ana Margarita González', 'Ana María Córdoba Barahona', 'Ana María Ramírez', 'Ana María Ruiz Perea', 'Ana Milena Muñoz de Gaviria', 'Andrea Aldana', 'Andrea Padilla Villarraga', 

In [12]:
# Contar columnas 
len(corpus_completo[(corpus_completo['Fecha_formateada'] < pd.to_datetime('2019-01-01'))])


len(corpus_completo.query("Fecha_formateada < '2021-01-01' and Fecha_formateada > '2019-12-31' and Diario == 'El Tiempo'"))
len(corpus_completo.query("Fecha_formateada < '2020-01-01' and Fecha_formateada > '2018-12-31'"))
len(corpus_completo.query("Diario == 'El Espectador'"))
len(corpus_completo)


13342

In [13]:
# Contar autores
corpus_filtrado = corpus_completo.query("Fecha_formateada < '2020-01-01' and Fecha_formateada > '2018-12-31' and Diario == 'Semana'")
corpus_filtrado = corpus_completo.query("Fecha_formateada < '2021-01-01' and Fecha_formateada > '2019-12-31'")
corpus_filtrado = corpus_completo.query("Diario == 'El Tiempo'")

len(corpus_filtrado['Autor'].unique().tolist())

245

In [14]:
# Contar palabras totales en la columna 'Texto'
num_palabras = corpus_completo['Texto'].apply(lambda x: len(str(x).split()))

In [15]:
num_palabras.sum()

8376957

In [16]:
num_palabras.describe()

count    13342.000000
mean       627.863664
std        210.554558
min        128.000000
25%        523.000000
50%        590.000000
75%        682.000000
max       5223.000000
Name: Texto, dtype: float64

In [17]:
num_palabras.groupby(corpus_completo['Diario']).describe()

,count,mean,std,min,25%,50%,75%,max
Diario,,,,,,,,
El Espectador,10519.0,613.876794,208.193068,153.0,516.00,574.0,666.0,5223.0
El Tiempo,2001.0,634.930535,157.632508,332.0,586.00,629.0,668.0,1813.0
Semana,822.0,789.648418,275.213383,128.0,608.25,736.0,936.0,2256.0


In [18]:
# Contar palabras distintas en la columna 'Texto'
texto_unido = ' '.join(corpus_completo['Texto'].dropna().astype(str).tolist())
palabras_distintas = set(texto_unido.split())

In [19]:
len(palabras_distintas)

336020

In [20]:
corpus_completo.groupby(pd.Grouper(key='Fecha_formateada', axis=0, freq='ME')).count()


,Diario,Autor,Fecha,Título,Texto,Vínculo
Fecha_formateada,,,,,,
2018-01-31,533,533,533,533,533,533
2018-02-28,476,476,476,476,476,476
2018-03-31,453,453,453,453,453,453
2018-04-30,479,479,479,479,479,479
2018-05-31,558,558,558,558,558,558
2018-06-30,362,362,362,362,362,362
2018-07-31,284,284,284,284,284,284
2018-08-31,372,372,372,372,372,372
2018-09-30,409,409,409,409,409,409


In [22]:
corpus_completo.groupby('Autor').count()

,Diario,Fecha,Título,Texto,Vínculo,Fecha_formateada
Autor,,,,,,
Abdón Espinosa Valderrama,2,2,2,2,2,2
Abel Veiga Copo,5,5,5,5,5,5
Adolfo León Atehortúa Cruz,11,11,11,11,11,11
Adolfo Meisel Roca,37,37,37,37,37,37
Adolfo Zableh Durán,53,53,53,53,53,53
...,...,...,...,...,...,...
Óscar Alarcón,110,110,110,110,110,110
Óscar Domínguez Giraldo,21,21,21,21,21,21
Óscar Rodrigo Campo Hurtado,1,1,1,1,1,1


['hola', 'mundo.']

# Transformación a archivos txt

In [110]:
corpus = corpus_completo.drop('Fecha_formateada', axis=1)
# Crear numeración de las filas
corpus['ID'] = range(1, len(corpus) + 1)

In [111]:
# Llevar dataframe a excel
corpus.to_excel('Corpus_completo.xlsx', index=False)

c:\Users\karen\anaconda3\Lib\site-packages\xlsxwriter\workbook.py:124: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.createtime = datetime.utcnow()
c:\Users\karen\anaconda3\Lib\site-packages\xlsxwriter\core.py:121: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date = self.properties.get("created", datetime.utcnow())
c:\Users\karen\anaconda3\Lib\site-packages\xlsxwriter\core.py:138: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date = self.properties.get("created", datetime.utcnow())


In [114]:
# Nombre de la carpeta donde se guardarán los archivos
carpeta_salida = "archivos_txt"

# Crear la carpeta si no existe
os.makedirs(carpeta_salida, exist_ok=True)


for index, row in corpus.iterrows():
    # Crear el nombre del archivo: "ID_Diario.txt"
    fila = str(row['ID']).zfill(5) 
    nombre_archivo = f"{fila}_{row['Diario']}.txt"
    
    # Ruta completa del archivo (dentro de la carpeta)
    ruta_archivo = os.path.join(carpeta_salida, nombre_archivo)
    
    # Contenido del archivo (aquí guardamos todas las columnas como texto separado por tabulaciones)
    contenido = "\n".join([str(row[col]) for col in ['ID', 'Diario','Título', 'Fecha', 'Autor', 'Texto']])
    
    # Escribir el archivo
    with open(ruta_archivo, 'w', encoding='utf-8') as f:
        f.write(contenido)
    
print(f"Archivo guardado: {ruta_archivo}")

Archivo guardado: archivos_txt\13342_Semana.txt


# Preprocesamiento

https://spacy.io/models/es

In [79]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---- ----------------------------------- 4.7/42.3 MB 35.7 MB/s eta 0:00:02
     ------------------ -------------------- 20.4/42.3 MB 61.5 MB/s eta 0:00:01
     --------------------------------------  41.4/42.3 MB 77.5 MB/s eta 0:00:01
     --------------------------------------- 42.3/42.3 MB 69.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [55]:
# Preprocesamiento de los datos

# Cargar modelo de lenguaje en español de Spacy, el modelo más pequeño
#nlp = spacy.load("es_core_news_sm")

# Cargar modelo de lenguaje en español de Spacy, el modelo mediano
nlp = spacy.load("es_core_news_md")


# Modelo que incluye transformers 
#nlp = spacy.load("es_dep_news_trf")

## Función de preprocesamiento 

In [56]:
def preprocesar_texto(texto):
    # Limpieza
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE) # eliminar URLs
    texto = re.sub(r'\S+@\S+', '', texto) # eliminar correos electrónicos
    texto = re.sub(r'[^\w\sáéíóúñÁÉÍÓÚÑ]', ' ', texto) # mantener solo caracteres alfanuméricos y acentos
    
    # Procesamiento con spaCy tokenizacion
    doc = nlp(texto)
    
    # Lematización y filtrado
    tokens = [
        token.text for token in doc
        if not token.is_punct 
        and not token.is_stop
        and not token.is_space
        and token.pos_ in ['NOUN', 'VERB', 'PROPN','ADJ', 'NUM']  # Tomar solo sustantivos, verbos, nombres propios, adjetivos y numerales
        and len(token.lemma_) > 3  # Eliminar palabras muy cortas
        and token.text.isalpha()  # Solo palabras alfabéticas
    ]
    
    return " ".join(tokens)

In [151]:
%pip install pyldavis


  Using cached pyLDAvis-3.4.1-py3-none-any.whl.metadata (4.2 kB)
Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)


# Texto Completo LDA

## Preprocesamiento y representación vectorial

In [7]:
# Para ver las stopwords en español
print(STOP_WORDS)

{'sí', 'tus', 'hago', 'qué', 'solos', 'tener', 'será', 'nuevo', 'podemos', 'son', 'estamos', 'queremos', 'va', 'podeis', 'eso', 'los', 'muchas', 'lleva', 'o', 'hicieron', 'qeu', 'tuyos', 'mediante', 'excepto', 'cuando', 'tenga', 'cualquier', 'este', 'diez', 'por', 'ésta', 'ésas', 'ellos', 'mismo', 'debe', 'deben', 'da', 'nada', 'está', 'cierto', 'bien', 'segundo', 'cuánta', 'nuestras', 'pasada', 'podrán', 'tendrán', 'peor', 'cuál', 'les', 'poco', 'parece', 'uno', 'tuvo', 'sin', 'expresó', 'nuestro', 'seis', 'aquel', 'tanto', 'saber', 'aquella', 'partir', 'tercera', 'podrían', 'así', 'ningunos', 'encuentra', 'habrá', 'consiguen', 'estaba', 'también', 'éstos', 'manera', 'podrá', 'usa', 'ahi', 'mucho', 'te', 'sólo', 'estas', 'tuya', 'soy', 'hacerlo', 'detras', 'sera', 'poner', 'mi', 'pues', 'tendrá', 'eran', 'fuimos', 'ahí', 'igual', 'existe', 'ésa', 'despacio', 'he', 'usas', 'cuáles', 'quedó', 'yo', 'ya', 'habia', 'todo', 'entre', 'respecto', 'fue', 'esa', 'primeros', 'dijo', 'siempre', 

In [57]:
# Añadir stopwords 
nlp.Defaults.stop_words |= {"tola","maruja","gazapera"}

In [65]:
# Procesar los primeros 1000 textos de columnas, eliminando urls, stopwords del diccionario de Spacy y lematizando
procesar = False
if procesar:
    columnas_procesadas = corpus_completo['Texto'].astype(str).apply(preprocesar_texto)


In [66]:
# Guardar en archivo JSON el texto procesado para evitar correrlo de nuevo
with open('columnas_procesadas.json', 'w', encoding='utf-8') as f:
    json.dump(columnas_procesadas.tolist(), f, ensure_ascii=False, indent=4)  

In [58]:
# Cargar el texto procesado desde el archivo JSON
with open('columnas_procesadas.json', 'r', encoding='utf-8') as f:
    columnas_procesadas = pd.Series(json.load(f))

In [59]:
longitudes = columnas_procesadas.apply(lambda x: len(x.split()))

print(longitudes.describe())

count    13344.000000
mean       252.886016
std         90.060471
min         53.000000
25%        209.000000
50%        237.000000
75%        275.000000
max       2259.000000
dtype: float64


In [60]:
def eliminar_bigrama(texto):
    return texto.replace("email protected", "")
columnas_procesadas = columnas_procesadas.apply(eliminar_bigrama)

Para la representación se puede matriz término documento por frecuencia y matriz con ponderación tf-idf, consideraré los bigramas también.

El parametro max_df indica ignorar palabras que aparecen en un porcentaje de documentos mayor al dado (ignorar palabras muy frecuentes). El min_df es la frecuencia mínima en documentos, para eliminar palabras muy raras.

In [61]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 4), max_df=0.7, min_df=10, max_features=15000)
matriz_columnas = tfidf_vectorizer.fit_transform(columnas_procesadas)

In [62]:
matriz_columnas.shape

(13344, 12845)

## Modelado Tópicos LDA
https://nbviewer.org/github/bmabey/pyLDAvis/blob/master/notebooks/LDA%20model.ipynb#topic=0&lambda=1&term= Para visualizar

In [75]:
# Función para mostrar los tópicos
def mostrar_topicos(modelo, vectorizer, n_palabras=10):
    palabras = vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico #{idx}:")
        print(", ".join([palabras[i] for i in topico.argsort()[:-n_palabras - 1:-1]]))
        print()

In [64]:
# Modelado de tópicos con LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(matriz_columnas)

mostrar_topicos(lda, tfidf_vectorizer)

Tópico #0:
donald trump, casa blanca, presidente trump, guerra mundial, política exterior, presidente unidos, medios comunicación, reino unido, unión europea, corea norte

Tópico #1:
frutas verduras, santo domingo, real madrid, desarrollo personalidad, libre desarrollo, duras penas, libre desarrollo personalidad, teatro santo, teatro santo domingo, garcía lorca

Tópico #2:
iván duque, centro democrático, álvaro uribe, corte suprema, gustavo petro, vargas lleras, presidente duque, juan manuel, uribe vélez, juan manuel santos

Tópico #3:
medios comunicación, presidente duque, libertad expresión, dosis mínima, lleras restrepo, carlos lleras, juan pablo, libertad prensa, investigador dejusticia, dejusticia profesor

Tópico #4:
lópez obrador, memoria histórica, nacional memoria, centro nacional, centro nacional memoria, nacional memoria histórica, centro nacional memoria histórica, cambio climático, manuel lópez, manuel lópez obrador

Tópico #5:
líderes sociales, derechos humanos, conflicto

In [76]:
vis_data = pyLDAvis.lda_model.prepare(lda, matriz_columnas, tfidf_vectorizer)

# Guardar como HTML y abrir en navegador
pyLDAvis.save_html(vis_data, 'lda_visualization.html')
import webbrowser
webbrowser.open('lda_visualization.html')

True

## BERTopic

https://maartengr.github.io/BERTopic/index.html#common Documentación del BERTopic

https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased 

https://colab.research.google.com/drive/18arPPe50szvcCp_Y6xS56H2tY0m-RLqv?usp=sharing#scrollTo=VUNsYrCETNar  Colab con ejemplo de uso


In [ ]:
%pip install bertopic

In [ ]:
%pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



In [30]:
%pip install hf_xet

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 26.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [43]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [66]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline

In [67]:
sentence_model = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

In [38]:
topic_model = BERTopic(language="spanish",
    top_n_words=10,          # Palabras por tópico
    min_topic_size=20,       # Tamaño mínimo del cluster
    nr_topics=50,        # Número de tópicos
    calculate_probabilities=True,
    vectorizer_model=TfidfVectorizer(stop_words=list(STOP_WORDS)),
    embedding_model= sentence_model 
    )

Tarda 11 minutos en correr.

In [39]:
texto_seleccionado = corpus_completo['Texto'].astype(str).tolist()
#texto_seleccionado = columnas_procesadas

CORRER = True
if CORRER:
    topics, probs = topic_model.fit_transform(texto_seleccionado)

In [44]:
# PRECAUCION AL CORRER ESTO
# Guardar el modelo entrenado y su información
with open('topics_bert.json', 'w', encoding='utf-8') as f:
    json.dump(topics, f, ensure_ascii=False, indent=4)
with open('probs_bert.json', 'w', encoding='utf-8') as f:
    json.dump(probs.tolist(), f, ensure_ascii=False, indent=4)
with open('topic_model.pkl', 'wb') as f:
    pickle.dump(topic_model, f)

In [68]:
# PRECAUCION AL CORRER ESTO
# Leer modelo, probabilidades y tópicos
with open('topics_bert.json', 'r', encoding='utf-8') as f:
    topics = json.load(f) 
with open('probs_bert.json', 'r', encoding='utf-8') as f:
    probs = json.load(f)
with open('topic_model.pkl', 'rb') as f:
    topic_model = pickle.load(f)

In [69]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,7083,-1_país_colombia_gobierno_años,"[país, colombia, gobierno, años, política, pre...",[Me parece que el doctor Vargas Lleras se vuel...
1,0,1219,0_economía_colombia_empresas_gobierno,"[economía, colombia, empresas, gobierno, país,...",[La economía lleva cuatro años estancada. En e...
2,1,525,1_virus_coronavirus_pandemia_covid,"[virus, coronavirus, pandemia, covid, salud, 1...","[En un hecho inédito para la humanidad, por pr..."
3,2,358,2_vida_mundo_casa_años,"[vida, mundo, casa, años, tiempo, amor, dios, ...",[¿Son entonces las mujeres culpables del maltr...
4,3,317,3_educación_universidad_estudiantes_universidades,"[educación, universidad, estudiantes, universi...",[En el año 1993 el expresidente César Gaviria ...
5,4,291,4_colombia_presidente_gobierno_país,"[colombia, presidente, gobierno, país, polític...",[Debo empezar esta columna diciendo que no soy...
6,5,281,5_paz_farc_santrich_guerrilla,"[paz, farc, santrich, guerrilla, guerra, jep, ...",[El lunes el país vio atónito cómo el CTI capt...
7,6,235,6_fútbol_jugadores_gol_equipo,"[fútbol, jugadores, gol, equipo, goles, equipo...","[La economía devastada, las fuerzas de ocupaci..."
8,7,222,7_música_arte_musical_teatro,"[música, arte, musical, teatro, festival, arti...",[El Ibagué Festival es un nuevo espacio cultur...
9,8,208,8_venezuela_maduro_guaidó_venezolano,"[venezuela, maduro, guaidó, venezolano, chávez...",[Venezuela es tendencia en todos los medios na...


In [70]:
topic_model.get_topic(2)

[('vida', 0.008060417730403812),
 ('mundo', 0.0050532043486178424),
 ('casa', 0.004723487294607641),
 ('años', 0.004512426428302214),
 ('tiempo', 0.00434080702148459),
 ('amor', 0.004197519672360857),
 ('dios', 0.004037030287553795),
 ('cosas', 0.00395077303627431),
 ('gente', 0.003872026684822089),
 ('alguien', 0.0038632782375553607)]

In [71]:
topic_model.representative_docs_[2]

['¿Son entonces las mujeres culpables del maltrato por no denunciar? ¡No, señores!. Sin lugar a dudas el 100 por ciento de la culpa recae sobre nosotros, los hombres. Los hombres que las tenemos muertas del miedo viviendo en un país que ocupa el puesto 14 en el mundo entre los países con más víctimas de ese flagelo.\n\nFaltaban unos diez minutos para que mi reloj marcara las ocho de la noche. Días antes, me había ofrecido a cocinar un arroz de mariscos para la familia de mi novia en la cena del 31 de diciembre. Aun cuando aprendí esa receta de mi madre, y me la sé de memoria, quise bañarme y vestirme con tiempo para hacer lo mío sin afán, y lograr que el plato estuviera a la altura de las expectativas de mis suegros.\n\nSalí del cuarto ya vestido y perfumado para dirigirme a la cocina de la casa en donde estábamos hospedados. Apenas empezaba a caminar hacia allá, cuando los ladridos estruendosos de unos perros de la propiedad de al lado me hicieron detener la marcha pensando que algo e

In [72]:
import plotly.express as px

# Obtener información de tópicos
topic_info = topic_model.get_topic_info()

# Filtrar outliers (tópico -1)
topic_info_filtered = topic_info[topic_info['Topic'] != -1]

# Gráfico de barras interactivo
fig = px.bar(topic_info_filtered,
             x='Topic',
             y='Count',
             color='Name',
             hover_data=['Representation'],
             title='Distribución de Tópicos',
             labels={'Count': 'N° Columnas', 'Name': 'Palabras Clave'},
             height=600)
fig.update_layout(xaxis={'type': 'category'})
fig.show()

¿Cómo forzar que todas las columnas queden en un tópico?